In [ ]:
import scipy
import pickle
!pip install spectral

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 6.2 MB/s eta 0:00:00


In [ ]:
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.src.utils import np_utils
import tensorflow as tf

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

from plotly.offline import init_notebook_mode

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral

import torch
from torch.utils.data import DataLoader,TensorDataset
import math
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import numpy as np
from torch.autograd import Function

init_notebook_mode(connected=True)
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loading

In [ ]:
## GLOBAL VARIABLES
test_ratio = 0.7
windowSize = 25

In [ ]:
def loadData():
    data = scipy.io.loadmat('/content/drive/MyDrive/Indian_pines_corrected.mat')['indian_pines_corrected']
    labels = scipy.io.loadmat('/content/drive/MyDrive/Indian_pines_gt.mat')['indian_pines_gt']

    return data, labels

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
X, y = loadData()
X.shape, y.shape

((145, 145, 200), (145, 145))

In [ ]:
K = X.shape[2]

In [ ]:
K = 15
X,pca = applyPCA(X,numComponents=K)

X.shape

(145, 145, 15)

In [ ]:
X, y = createImageCubes(X, y, windowSize=windowSize)

X.shape, y.shape

((10249, 25, 25, 15), (10249,))

In [ ]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((3074, 25, 25, 15), (7175, 25, 25, 15), (3074,), (7175,))

# Model and Training

In [ ]:
Xtrain = Xtrain.reshape(-1, windowSize, windowSize, K, 1)
Xtrain.shape

(3074, 25, 25, 15, 1)

In [ ]:
ytrain = np_utils.to_categorical(ytrain,num_classes=16)
ytrain.shape

(3074, 16)

In [ ]:
Xtrain, ytrain, Xtest, ytest = torch.tensor(Xtrain),torch.tensor(ytrain),torch.tensor(Xtest),torch.tensor(ytest)
Xtrain = Xtrain.reshape(-1,1,25,25,15)
Xtest = Xtest.reshape(-1,1,25,25,15)

In [ ]:
S = windowSize
L = K
output_units = 16

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Function

class Round(Function):
    @staticmethod
    def forward(self, input):
        sign = torch.sign(input)
        output = sign * torch.floor(torch.abs(input) + 0.5)
        return output

    @staticmethod
    def backward(self, grad_output):
        grad_input = grad_output.clone()
        return grad_input

class QuantActivation(nn.Module):
    def __init__(self):
        super(QuantActivation, self).__init__()
        self.a_bits = nn.Parameter(torch.Tensor(torch.round(1+5*torch.rand(1))),requires_grad=True)

    def round(self, input):
        output = Round.apply(input)
        return output

    def forward(self, input):
        output = torch.clamp(input, -1, 1)

        scale = 1 / float(2 ** self.a_bits - 1)

        output = self.round(output / scale) * scale

        return output


In [ ]:
class HardBinaryConv2D(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(HardBinaryConv2D, self).__init__()
        self.stride = stride
        self.padding = padding
        self.number_of_weights = in_channels * out_channels * kernel_size[0] * kernel_size[1]
        self.shape = (out_channels, in_channels, kernel_size[0], kernel_size[1])
        self.weights = nn.Parameter(torch.rand((out_channels,in_channels,kernel_size[0],kernel_size[1])) * 0.001, requires_grad=True)

    def forward(self, x):
        real_weights = self.weights.view(self.shape)
        scaling_factor = torch.mean(torch.mean(abs(real_weights), dim=3, keepdim=True), dim=2, keepdim=True)
        scaling_factor = scaling_factor.detach()
        binary_weights_no_grad = scaling_factor * torch.sign(real_weights)
        cliped_weights = torch.clamp(real_weights, -1.0, 1.0)
        binary_weights = binary_weights_no_grad.detach() - cliped_weights.detach() + cliped_weights
        y = F.conv2d(x, binary_weights, stride=self.stride, padding=self.padding)
        return y



In [ ]:
class HardBinaryConv3D(torch.nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        self.stride = stride
        self.padding = padding
        self.kernel_size = kernel_size
        self.weights = torch.nn.Parameter(torch.rand((out_channels, in_channels, kernel_size[0], kernel_size[1], kernel_size[2])) * 0.001, requires_grad=True)

    def forward(self, x):
        real_weights = self.weights
        scaling_factor = torch.mean(torch.mean(abs(real_weights), dim=4, keepdim=True), dim=3, keepdim=True)
        scaling_factor = scaling_factor.detach()
        binary_weights_no_grad = scaling_factor * torch.sign(real_weights)
        cliped_weights = torch.clamp(real_weights, -1.0, 1.0)
        binary_weights = binary_weights_no_grad.detach() - cliped_weights.detach() + cliped_weights
        y = torch.nn.functional.conv3d(x.double(), binary_weights, stride=self.stride, padding=self.padding)
        return y



In [ ]:
class HybridSn(nn.Module):
    def __init__(self, num_classes = 16):
        super(HybridSn,self).__init__()
        self.conv1 = HardBinaryConv3D(1, 8, kernel_size=(3, 3, 7), stride=1, padding=0)
        self.saq1 = QuantActivation()
        self.conv2 = HardBinaryConv3D(8, 16, kernel_size=(3, 3, 5), stride=1, padding=0)
        self.saq2 = QuantActivation()
        self.conv3 = HardBinaryConv3D(16, 32, kernel_size=(3, 3, 3), stride=1, padding=0)
        self.saq3 = QuantActivation()
        self.conv4 = HardBinaryConv2D(96, 64, kernel_size=(3, 3), stride=1, padding=0)
        self.saq4 = QuantActivation()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(18496, 256, bias=False)
        self.fc2 = nn.Linear(256, 128, bias=False)
        self.fc3 = nn.Linear(128, num_classes, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.saq1(x)
        x = self.conv2(x)
        x = self.saq2(x)
        x = self.conv3(x)
        x = self.saq3(x)
        x = torch.reshape(x,(x.shape[0],x.shape[1]*x.shape[4],x.shape[2],x.shape[3]))
        x = self.conv4(x)
        x = self.saq4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


In [ ]:
import numpy as np

def batch_generator(data, labels, batch_size=32, shuffle=True):

    data_size = len(data)
    indices = np.arange(data_size)
    batch_d = []
    batch_l = []
    for start_idx in range(0, data_size - batch_size + 1, batch_size):
        batch_indices = indices[start_idx:start_idx + batch_size]
        batch_data = data[batch_indices]
        batch_labels = labels[batch_indices]
        batch_d.append(batch_data)
        batch_l.append(batch_labels)
    return [batch_d,batch_l]


batch_size = 32
[batch_d,batch_l] = batch_generator(Xtrain, ytrain, batch_size)
batch_l[0].shape


torch.Size([32, 16])

In [ ]:
[testX_batch,testY_batch] = batch_generator(Xtest,ytest,batch_size)

In [ ]:
testY_batch[0].shape

torch.Size([32])

In [ ]:
model = HybridSn(num_classes=16)

In [ ]:
numepochs = 10
model = model.double()

def trainTheModel():
  lossfun= nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01,
                                weight_decay=5e-4, eps=1e-8, betas=(0.9, 0.999))
  maxTrainAcc = 0
  losses   = torch.zeros(numepochs)
  trainAcc = []
  testAcc  = []
  for epochi in range(numepochs):
    model.train()
    batchAcc  = []
    batchLoss = []
    for batch_data, batch_labels in zip(batch_d,batch_l):
      yHat = model(batch_data.float())
      loss = lossfun(yHat,batch_labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      ypred = torch.argmax(yHat,axis=1)
      yt = torch.argmax(batch_labels,axis=1)
      ypred = ypred.detach().numpy()
      yt = yt.detach().numpy()
      batchLoss.append(loss.item())
      batchAcc.append( 100*np.mean((1*(ypred == yt))))

    trainAcc.append( np.mean(batchAcc) )


    losses[epochi] = np.mean(batchLoss)
    print("Epoch: ",epochi," done, Loss: ",losses[epochi]," Acc: ",trainAcc[epochi])
    if(trainAcc[epochi]>maxTrainAcc):
      pickle.dump(model, open('/content/drive/My Drive/RL_Project/btpmodel.pkl', 'wb'))
      maxTrainAcc = trainAcc[epochi]
      print('Accuracy improved. Model saved')



In [ ]:
trainTheModel()


In [ ]:
bestmodel = pickle.load(open('/content/drive/My Drive/RL_Project/btpmodel.pkl', 'rb'))
bestmodel.eval()
testBAcc = []
for testd, testl in zip(testX_batch,testY_batch):
  yHat_t = bestmodel(testd.float())
  ypred_t = torch.argmax(yHat_t,axis=1)
  y_t = testl
  ypred_t = ypred_t.detach().numpy()
  y_t = y_t.detach().numpy()
  print(classification_report(y_t,ypred_t))
  testBAcc.append(100*np.mean((1*(ypred_t==y_t))))

              precision    recall  f1-score   support

         1.0       1.00      0.67      0.80         3
         2.0       1.00      0.67      0.80         3
         3.0       1.00      1.00      1.00         1
         4.0       1.00      1.00      1.00         4
         5.0       1.00      1.00      1.00         3
         6.0       1.00      1.00      1.00         1
         7.0       1.00      1.00      1.00         1
         9.0       1.00      1.00      1.00         4
        10.0       0.86      1.00      0.92         6
        12.0       0.50      1.00      0.67         1
        13.0       1.00      1.00      1.00         5

    accuracy                           0.94        32
   macro avg       0.94      0.94      0.93        32
weighted avg       0.96      0.94      0.94        32

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         1
         1.0       1.00      0.86      0.92         7
         2.0       1.00 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to cont

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         1
         2.0       1.00      1.00      1.00         6
         3.0       0.00      0.00      0.00         0
         4.0       0.50      1.00      0.67         1
         5.0       1.00      1.00      1.00         3
         6.0       0.00      0.00      0.00         1
         9.0       1.00      1.00      1.00         3
        10.0       1.00      0.90      0.95        10
        11.0       1.00      1.00      1.00         1
        12.0       1.00      1.00      1.00         1
        13.0       1.00      1.00      1.00         2
        14.0       1.00      1.00      1.00         3

    accuracy                           0.94        32
   macro avg       0.79      0.83      0.80        32
weighted avg       0.95      0.94      0.94        32



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

         1.0       1.00      0.75      0.86         4
         2.0       0.67      1.00      0.80         2
         3.0       1.00      1.00      1.00         2
         4.0       1.00      1.00      1.00         1
         5.0       1.00      1.00      1.00         3
         7.0       1.00      1.00      1.00         3
         8.0       0.00      0.00      0.00         1
         9.0       1.00      1.00      1.00         2
        10.0       0.75      1.00      0.86         3
        11.0       1.00      1.00      1.00         1
        13.0       1.00      1.00      1.00         6
        14.0       1.00      1.00      1.00         3
        15.0       1.00      1.00      1.00         1

    accuracy                           0.94        32
   macro avg       0.88      0.90      0.89        32
weighted avg       0.92      0.94      0.93        32



...

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         6
         2.0       0.50      1.00      0.67         2
         3.0       1.00      0.67      0.80         3
         4.0       1.00      1.00      1.00         1
         5.0       1.00      1.00      1.00         1
         6.0       1.00      1.00      1.00         1
         7.0       1.00      1.00      1.00         1
         9.0       1.00      1.00      1.00         1
        10.0       1.00      1.00      1.00         4
        11.0       1.00      0.50      0.67         2
        12.0       1.00      1.00      1.00         1
        13.0       0.83      1.00      0.91         5
        14.0       1.00      1.00      1.00         3
        15.0       0.00      0.00      0.00         1

    accuracy                           0.91        32
   macro avg       0.88      0.87      0.86        32
weighted avg       0.91      0.91      0.89        32



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to cont

              precision    recall  f1-score   support

         1.0       1.00      0.80      0.89         5
         4.0       0.00      0.00      0.00         1
         5.0       1.00      1.00      1.00         5
         7.0       1.00      1.00      1.00         3
         9.0       0.50      1.00      0.67         1
        10.0       1.00      1.00      1.00         6
        11.0       1.00      0.50      0.67         2
        13.0       0.88      1.00      0.93         7
        14.0       1.00      1.00      1.00         2
        15.0       0.00      0.00      0.00         0

    accuracy                           0.91        32
   macro avg       0.74      0.73      0.72        32
weighted avg       0.93      0.91      0.91        32

              precision    recall  f1-score   support

         1.0       1.00      0.50      0.67         6
         2.0       0.67      1.00      0.80         2
         4.0       1.00      0.67      0.80         3
         5.0       1.00 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to cont

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      1.00      1.00         4
         2.0       1.00      1.00      1.00         3
         4.0       0.75      1.00      0.86         3
         5.0       1.00      1.00      1.00         1
         6.0       0.00      0.00      0.00         1
         7.0       1.00      1.00      1.00         1
         9.0       1.00      0.75      0.86         4
        10.0       1.00      1.00      1.00        12
        12.0       1.00      1.00      1.00         1
        13.0       1.00      1.00      1.00         2

    accuracy                           0.94        32
   macro avg       0.80      0.80      0.79        32
weighted avg       0.95      0.94      0.94        32

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         3
         2.0       1.00      1.00      1.00         1
         5.0       0.50 

In [ ]:
print("Overall test accuracy: ",np.mean(testBAcc))

Overall test accuracy: 99.21457585265712
